In [1]:
# Install the python package for google gen AI
!pip install -q -U google-generativeai

In [2]:
# Import necessary packages and define helper function to render output of models to markdown
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
from google.colab import userdata

In [3]:
# Grab API Key from your secrets, add it to secrets if its not there (on the left hand panel)
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Configure genai to use your API Key
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
# access Google Drive files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

SEED = 9999

In [6]:
data = pd.read_csv('/content/drive/MyDrive/MEXA/mental_health_wearable_data.csv')

X = data[['Heart_Rate_BPM', 'Sleep_Duration_Hours', 'Physical_Activity_Steps']]
y = data['Mood_Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [7]:
wearablemodel = LinearRegression()
wearablemodel.fit(X_train, y_train)

LinearRegression()

In [8]:
y_pred = wearablemodel.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 6.550228122268496


In [11]:
# Select a model and instantiate a GenerativeModel
model = genai.GenerativeModel('gemini-1.5-pro')

In [12]:
with open('/content/drive/MyDrive/MEXA/Carrie.txt', 'r') as file:
    carrie_prompt = file.read()

In [13]:
# You can now use the model defind about to generate content base on inputs
response = model.generate_content([carrie_prompt, f"(wearable: Mood_Rating = {y_pred[-1]}/9)",
    "I'm 19, he's 25. He's not very considerate of how I feel but says he cares about me and loves me."
    ])

In [14]:
chat = model.start_chat(
    history=[
        {"role": "user", "parts": carrie_prompt},
        {"role": "model", "parts": "Hey there! What's up? How's it going? Anything you wanna chat about? I'm all ears. 😊"},
    ]
)

In [15]:
response = chat.send_message("(wearable: Mood_Rating = 3/9) I'm 19, he's 25. He's not very considerate of how I feel but says he cares about me and loves me.")

In [16]:
# This makes the response easier to read
to_markdown(response.text)

> That sounds tricky.  It's tough when someone says they care but their actions don't quite match up.  It's important to feel heard and respected in a relationship. Have you tried talking to him about how his actions make you feel? Sometimes people aren't aware of how they're coming across.
> 
> 
